In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
# df = pd.read_csv("../input/ss3-hackathon-online-signal-processing/SN_Test/test_data_1.csv")
df = pd.read_csv("../input/ss3-hackathon-online-signal-processing/SN_Train/normal_1411_1.csv")
# df = pd.read_csv("../input/ss3-hackathon-online-signal-processing/SN_Train/normal_576_1.csv")
# df = pd.read_csv("../input/ss3-hackathon-online-signal-processing/SN_Train/normal_576_10.csv")
# df = pd.read_csv("../input/ss3-hackathon-online-signal-processing/SN_Train/unbalanceM1_1411_1.csv")

In [ ]:
df.head()
# df.columns
# df.info()
# df['V1A'] = pd.to_numeric(df['V1A'], errors='coerce')

# Read file
functional, clean data, remove DC Bias, reduce noise by lfilter, transpose_to( #files, #sample_size, #components )

In [ ]:
from scipy.signal import lfilter

TRAIN_FOLDER = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Train/'
TEST_FOLDER = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Test/'

def do_lfilter(y):
    n = 15  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    yy = lfilter(b, a, y)
    return yy

def folder_tosignal(folder):
    md = 1e9
    fi2 = True
    for f in os.listdir(folder):
        df = pd.read_csv(folder + f)
        df = df[df['waveform'].str.contains('t0')==False]
        df = df[df['waveform'].str.contains('delta t')==False]
        df = df[df['waveform'].str.contains('time')==False]
        df = df[df['waveform'].str.contains('wave')==False]
        df = df.drop(['waveform'], axis=1)
        fi = True
        for col in df.columns:
            df[col] = pd.to_numeric(df[col])
            signal = df[col].to_numpy()
            signal = signal - np.mean(signal) # Remove DC Bias
            signal = do_lfilter(signal) # Reduce Noise by lfilter
            signal_components = signal if fi else np.vstack((signal_components, signal)) # stack signal (10000+ sample) each sensor component (6)
            fi = False
        md = min(signal_components.shape[1], md) # select least sample size for all signals
        all_signals = signal_components if fi2 else np.dstack((all_signals[:, :md], signal_components[:, :md])) # stack motor signal (1000+)
        print(f'converting {f}: {all_signals.shape}')
        fi2 = False

    all_signals = np.transpose(all_signals, (2, 1, 0))
    return all_signals

train_signal = folder_tosignal(TRAIN_FOLDER)
test_signal = folder_tosignal(TEST_FOLDER)

In [ ]:
def get_file_name(folder):
    l = []
    for f in os.listdir(folder):
        l.append(f)
    return pd.DataFrame(l)

file_df = get_file_name(TEST_FOLDER)
file_df

In [ ]:
def get_file_num(folder):
    l = []
    for f in os.listdir(folder):
        l.append(f.split('_')[2].split('.')[0])
    return pd.DataFrame(l).astype('int64')

num_df = get_file_num(TEST_FOLDER)
num_df

In [ ]:
# train เก่า
TRAIN_FOLDER = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Train/'

test_file = ['normal_1411_1.csv', 'unbalanceM1_1411_1.csv', 'unbalanceM2_1411_1.csv']

x_train = np.array([])
y_train = []
md = 1e9
fi2 = True

for f in os.listdir(TRAIN_FOLDER):
    df = pd.read_csv(TRAIN_FOLDER + f)
    df = df[df['waveform'].str.contains('t0')==False]
    df = df[df['waveform'].str.contains('delta t')==False]
    df = df[df['waveform'].str.contains('time')==False]
    df = df[df['waveform'].str.contains('wave')==False]
    df = df.drop(['waveform'], axis=1)
    fi = True
    for col in df.columns:
        df[col] = pd.to_numeric(df[col])
        sig = df[col].to_numpy()
#         print(sig.shape)
        s = sig if fi else np.vstack((s, sig))
#         print(s.shape)
        fi = False
    c = f.split('_')[0]
    y_train.append(c)
#     print(s.shape, x_train.shape)
    md = min(s.shape[1], md)
#     print(md)
    x_train = s if fi2 else np.dstack((x_train[:, :md], s[:, :md]))
    print(f'converting {f}: {x_train.shape}')
    fi2 = False
#     x_train.append(s)

t = np.transpose(x_train, (2, 1, 0))

In [ ]:
# Test
TEST_FOLDER = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Test/'

x_test = np.array([])
md = 1e9
fi2 = True

for f in os.listdir(TEST_FOLDER):
    df = pd.read_csv(TEST_FOLDER + f)
    df = df[df['waveform'].str.contains('t0')==False]
    df = df[df['waveform'].str.contains('delta t')==False]
    df = df[df['waveform'].str.contains('time')==False]
    df = df[df['waveform'].str.contains('wave')==False]
    df = df.drop(['waveform'], axis=1)
    fi = True
    for col in df.columns:
        df[col] = pd.to_numeric(df[col])
        sig = df[col].to_numpy()
#         print(sig.shape)
        s = sig if fi else np.vstack((s, sig))
#         print(s.shape)
        fi = False
#     c = f.split('_')[0]
#     y_train.append(c)
#     print(s.shape, x_train.shape)
    md = min(s.shape[1], md)
#     print(md)
    x_test = s if fi2 else np.dstack((x_test[:, :md], s[:, :md]))
    print(f'converting {f}: {x_test.shape}')
    fi2 = False
#     x_train.append(s)

X_test = np.transpose(x_test, (2, 1, 0))

In [ ]:
X_test.shape

In [ ]:
t = np.transpose(x_train, (2, 1, 0))
tt = np.transpose(x_train, (1, 2, 0))

In [ ]:
N = train_signals.shape[1]
f_s = 10000
T = 1 / f_s

In [ ]:
N, f_s, T

In [ ]:
# the number of signals in the training set is 7352 
# and the number of signals in the test set is 2947 
# And each signal in the training and test set has a length of 128 samples 
# and 9 different components.
# (7352, 128, 9)
# (2947, 128, 9)
# My Signal
# (1000, 10000, 6) use t (2, 1, 0)
x_train.shape, t.shape, tt.shape

In [ ]:
X_train = t
X_train.shape

In [ ]:
y_train

In [ ]:
Y_train = np.array(y_train)

In [ ]:
dfy = pd.DataFrame(y_train)
dfy.iloc[:, 0].unique()

# Encode Y_train

In [ ]:
INPUT_FOLDER_TRAIN = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Train/'

def read_labels(folder):
    motors = []
    for file in os.listdir(folder):
        m = file.split('_')[0]
        motors.append(m)
    return np.array(motors)

train_labels = read_labels(INPUT_FOLDER_TRAIN)
train_labels

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['normal', 'unbalanceM1', 'unbalanceM2', 'unbalanceM3'])
Y_train = le.transform(train_labels)
Y_train

In [ ]:
Y_train, Y_train.shape

In [ ]:
np.save('X_train.npy', X_train)

In [ ]:
x = np.load('X_train.npy')
x.shape

In [ ]:
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
train_signals = X_train
train_labels = Y_train
test_signals = X_test

In [ ]:
TRAIN_FOLDER = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Train'
TEST_FOLDER = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Test'
x_train = np.array([])
rpm = []
y_train = []
test_file = ['normal_1411_1.csv', 'unbalanceM1_1411_1.csv', 'unbalanceM2_1411_1.csv']
fi2 = True
md = 100000
for f in test_file: # os.listdir(TRAIN_FOLDER):
    df = pd.read_csv(TRAIN_FOLDER + '/' + f)
    df = df[df['waveform'].str.contains('t0')==False]
    df = df[df['waveform'].str.contains('delta t')==False]
    df = df[df['waveform'].str.contains('time')==False]
    df = df[df['waveform'].str.contains('wave')==False]
    df = df.drop(['waveform'], axis=1)
    fi = True
    for col in df.columns:
        df[col] = pd.to_numeric(df[col])
        sig = df[col].to_numpy()
#         print(sig.shape)
        s = sig if fi else np.vstack((s, sig))
#         print(s.shape)
        fi = False
    c, r = f.split('_')[:2]
    rpm.append(r)
    y_train.append(c)
    print(s.shape, x_train.shape)
    md = min(s.shape[1], md)
#     print(md)
    x_train = s if fi2 else np.dstack((x_train[:, :md], s[:, :md]))
    fi2 = False
#     x_train.append(s)

t = x_train.reshape((3, 19993, 6))
x_train.shape, t.shape, s.shape, s.transpose().shape

In [ ]:
def read_signals(filename):
    df = pd.read_csv(filename)
    df = df[df['waveform'].str.contains('t0')==False]
    df = df[df['waveform'].str.contains('delta t')==False]
    df = df[df['waveform'].str.contains('time')==False]
    df = df[df['waveform'].str.contains('wave')==False]
    df = df.drop(['waveform'], axis=1)
    data = np.array(df, dtype=np.float32)
    print(f'converting {filename}: {data.shape}')
    return data

def read_labels(folder):
    motors = []
    for file in os.listdir(folder):
        m = f.split('_')[0]
    return np.array(motors)

# def read_labels(foldername):        
#     with open(filename, 'r') as fp:
#         activities = fp.read().splitlines()
#         activities = list(map(int, activities))
#     return np.array(motors)

INPUT_FOLDER_TRAIN = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Train/'
INPUT_FOLDER_TEST = '/kaggle/input/ss3-hackathon-online-signal-processing/SN_Train/'

train_signals, test_signals = [], []

md = 1e9

for input_file in os.listdir(INPUT_FOLDER_TRAIN):
    signal = read_signals(INPUT_FOLDER_TRAIN + input_file)
#     md = min(signal[0], md)
#     train_signals.append(signal[:md, :])
    train_signals.append(signal)
train_signals = np.transpose(np.array(train_signals), (1, 2, 0))

for input_file in os.listdir(INPUT_FOLDER_TEST):
    signal = read_signals(INPUT_FOLDER_TEST + input_file)
    test_signals.append(signal)
test_signals = np.transpose(np.array(test_signals), (1, 2, 0))

In [ ]:
train_labels

In [ ]:
train_signals

# Plot graph

In [ ]:
from scipy.fftpack import fft
from scipy.signal import welch
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
df['V1A'].count()

In [ ]:
%config Completer.use_jedi = False

In [ ]:
y

In [ ]:
# lfilter
from scipy.signal import lfilter

n = 15  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1
yy = lfilter(b, a, y)
plt.plot(x, yy, linewidth=2, linestyle="-", c="b")  # smooth by filter

In [ ]:
fig, ax = plt.subplots()
N = df['V1A'].count()
# N = 1000
T = 1.0 / 10000.0
x = np.linspace(0, 10, N)
y = df.iloc[:N, 2].to_numpy()
y = y - np.mean(y) # DC Offset !!!
ax.plot(x, y)
# ax.scatter(x, df.iloc[:N, 2])
plt.show()

fig, ax = plt.subplots()
x = np.linspace(0.0, N*T, N, endpoint=False)
yf = fft(yy)
xf = fftfreq(N, T)[:N//2]
plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.grid()
plt.show()

In [ ]:
def get_fft_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values
 
t_n = 10
N = df['V1A'].count()
# N = 1000
T = t_n / N
# T = 1 / f_s
# f_s = 10000
 
f_values, fft_values = get_fft_values(df.iloc[:N, 2].to_numpy(), T, N, f_s)
 
plt.plot(f_values, fft_values, linestyle='-', color='blue')
plt.xlabel('Frequency [Hz]', fontsize=16)
plt.ylabel('Amplitude', fontsize=16)
plt.title("Frequency domain of the signal", fontsize=16)
plt.show()

In [ ]:
f_values, psd_values = get_psd_values(df.iloc[:N, 2].to_numpy(), T, N, f_s)
 
plt.plot(f_values, psd_values, linestyle='-', color='blue')
plt.xlabel('Frequency [Hz]')
plt.ylabel('PSD [V**2 / Hz]')
plt.show()

In [ ]:
yf.dtype

# Copy preprocess

In [ ]:
def get_fft_values(y_values, T, N, f_s):
    f_values = np.linspace(0.0, 1.0/(2.0*T), N//2)
    fft_values_ = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values_[0:N//2])
    return f_values, fft_values

def get_psd_values(y_values, T, N, f_s):
    f_values, psd_values = welch(y_values, fs=f_s)
    return f_values, psd_values

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[len(result)//2:]

def get_autocorr_values(y_values, T, N, f_s):
    autocorr_values = autocorr(y_values)
    x_values = np.array([T * jj for jj in range(0, N)])
    return x_values, autocorr_values

In [ ]:
# Copied From http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
# Thank you Marcos Duarte

def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising',
                 kpsh=False, valley=False, show=False, ax=None):

    """Detect peaks in data based on their amplitude and other features.
    Parameters
    ----------
    x : 1D array_like
        data.
    mph : {None, number}, optional (default = None)
        detect peaks that are greater than minimum peak height.
    mpd : positive integer, optional (default = 1)
        detect peaks that are at least separated by minimum peak distance (in
        number of data).
    threshold : positive number, optional (default = 0)
        detect peaks (valleys) that are greater (smaller) than `threshold`
        in relation to their immediate neighbors.
    edge : {None, 'rising', 'falling', 'both'}, optional (default = 'rising')
        for a flat peak, keep only the rising edge ('rising'), only the
        falling edge ('falling'), both edges ('both'), or don't detect a
        flat peak (None).
    kpsh : bool, optional (default = False)
        keep peaks with same height even if they are closer than `mpd`.
    valley : bool, optional (default = False)
        if True (1), detect valleys (local minima) instead of peaks.
    show : bool, optional (default = False)
        if True (1), plot data in matplotlib figure.
    ax : a matplotlib.axes.Axes instance, optional (default = None).
    Returns
    -------
    ind : 1D array_like
        indeces of the peaks in `x`.
    Notes
    -----
    The detection of valleys instead of peaks is performed internally by simply
    negating the data: `ind_valleys = detect_peaks(-x)`
    
    The function can handle NaN's 
    See this IPython Notebook [1]_.
    References
    ----------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
    Examples
    --------
    >>> from detect_peaks import detect_peaks
    >>> x = np.random.randn(100)
    >>> x[60:81] = np.nan
    >>> # detect all peaks and plot data
    >>> ind = detect_peaks(x, show=True)
    >>> print(ind)
    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # set minimum peak height = 0 and minimum peak distance = 20
    >>> detect_peaks(x, mph=0, mpd=20, show=True)
    >>> x = [0, 1, 0, 2, 0, 3, 0, 2, 0, 1, 0]
    >>> # set minimum peak distance = 2
    >>> detect_peaks(x, mpd=2, show=True)
    >>> x = np.sin(2*np.pi*5*np.linspace(0, 1, 200)) + np.random.randn(200)/5
    >>> # detection of valleys instead of peaks
    >>> detect_peaks(x, mph=0, mpd=20, valley=True, show=True)
    >>> x = [0, 1, 1, 0, 1, 1, 0]
    >>> # detect both edges
    >>> detect_peaks(x, edge='both', show=True)
    >>> x = [-2, 1, -2, 2, 1, 1, 3, 0]
    >>> # set threshold = 2
    >>> detect_peaks(x, threshold = 2, show=True)
    """

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd) \
                    & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    if show:
        if indnan.size:
            x[indnan] = np.nan
        if valley:
            x = -x
        _plot(x, mph, mpd, threshold, edge, valley, ax, ind)

    return ind


In [ ]:
def get_first_n_peaks(x,y,no_peaks=5):
    x_, y_ = list(x), list(y)
    if len(x_) >= no_peaks:
        return x_[:no_peaks], y_[:no_peaks]
    else:
        missing_no_peaks = no_peaks-len(x_)
        return x_ + [0]*missing_no_peaks, y_ + [0]*missing_no_peaks
    
def get_features(x_values, y_values, mph):
    indices_peaks = detect_peaks(y_values, mph=mph)
    peaks_x, peaks_y = get_first_n_peaks(x_values[indices_peaks], y_values[indices_peaks])
    return peaks_x + peaks_y

def extract_features(dataset, T, N, f_s, denominator):
    percentile = 5
    list_of_features = []
    for signal_no in range(0, len(dataset)):
        features = []
        for signal_comp in range(0,dataset.shape[2]):
            signal = dataset[signal_no, :, signal_comp]
            
            signal_min = np.nanpercentile(signal, percentile)
            signal_max = np.nanpercentile(signal, 100-percentile)
            #ijk = (100 - 2*percentile)/10
            mph = signal_min + (signal_max - signal_min)/denominator
            
            features += get_features(*get_psd_values(signal, T, N, f_s), mph)
            features += get_features(*get_fft_values(signal, T, N, f_s), mph)
            features += get_features(*get_autocorr_values(signal, T, N, f_s), mph)
        list_of_features.append(features)
    return np.array(list_of_features)

# def extract_features_labels(dataset, labels, T, N, f_s, denominator):
#     percentile = 5
#     list_of_features = []
#     list_of_labels = []
#     for signal_no in range(0, len(dataset)):
#         features = []
#         list_of_labels.append(labels[signal_no])
#         for signal_comp in range(0,dataset.shape[2]):
#             signal = dataset[signal_no, :, signal_comp]
            
#             signal_min = np.nanpercentile(signal, percentile)
#             signal_max = np.nanpercentile(signal, 100-percentile)
#             #ijk = (100 - 2*percentile)/10
#             mph = signal_min + (signal_max - signal_min)/denominator
            
#             features += get_features(*get_psd_values(signal, T, N, f_s), mph)
#             features += get_features(*get_fft_values(signal, T, N, f_s), mph)
#             features += get_features(*get_autocorr_values(signal, T, N, f_s), mph)
#         list_of_features.append(features)
#     return np.array(list_of_features), np.array(list_of_labels)

from scipy.fftpack import fft
from scipy.signal import welch
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

denominator = 10
N = train_signal.shape[1]
f_s = 10000
T = 1 / f_s
X_train = extract_features(train_signal, T, N, f_s, denominator)
N = test_signal.shape[1]
X_test = extract_features(test_signal, T, N, f_s, denominator)
# X_train = extract_features_labels(train_signals, T, N, f_s, denominator)
# X_test = extract_features_labels(test_signals, T, N, f_s, denominator)

In [ ]:
X_train.shape, Y_train.shape, X_test.shape

# Improve Model

### split train-val for model improving

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
 
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, Y_train)
print("Accuracy on training set is : {}".format(clf.score(X_train, Y_train)))
Y_test_pred = clf.predict(X_test)
Y_test_pred

In [ ]:
!pip install -Uqq fastai

In [ ]:
!pip install fastai==2.1.4
!pip install fastcore==1.3.1

### fastai tabular

In [ ]:
from fastai.tabular.all import *

In [ ]:
df = pd.DataFrame(X_train)
df['label'] = pd.DataFrame(Y_train)
# df['label'] = pd.DataFrame(train_labels)
df

In [ ]:
cont_names = list(range(180))
# cont_names = [str(i) for i in list(range(180))]

# splits = RandomSplitter()(range_of(df))
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

to = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                   cont_names = cont_names,
                   y_names='label',
                   y_block = CategoryBlock,
                   splits=splits)

In [ ]:
to.xs.iloc[:2]

In [ ]:
dls = to.dataloaders(bs=64)
dls.show_batch()

In [ ]:
# learn = tabular_learner(dls, metrics=f1_macro)
learn = tabular_learner(dls, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(2, 0.001737800776027143)
# learn.fit_one_cycle(100, 0.001737800776027143)

In [ ]:
learn.show_results()

In [ ]:
test_df = pd.DataFrame(X_test)

In [ ]:
test_df.head()

In [ ]:
row, clas, probs = learn.predict(test_df.iloc[0])
row, clas, probs

In [ ]:
learn.dls.vocab

In [ ]:
dl = learn.dls.test_dl(test_df)
preds, y = learn.get_preds(dl=dl)
y = torch.argmax(preds, dim=1)
y

In [ ]:
def tensor_todf(y):
    l = []
    for c in y.numpy():
        if c == 0: c = 'normal'
        elif c == 1: c = 'unbalanceM1'
        elif c == 2: c = 'unbalanceM2'
        elif c == 3: c = 'unbalanceM3'
        l.append(c)
    return pd.DataFrame(l)

p_df = tensor_todf(y) # learner
# p_df = tensor_todf(torch.tensor(Y_test_pred)) # forest
p_df

In [ ]:
# def predict_df(df):
#     l = []
#     for i in range(360):
#         print(i)
#         _, clas, _ = learn.predict(test_df.iloc[i])
#         c = clas.numpy()
#         if c == 0: c = 'normal'
#         elif c == 1: c = 'unbalanceM1'
#         elif c == 2: c = 'unbalanceM2'
#         elif c == 3: c = 'unbalanceM3'
#         l.append(c)
#     return pd.DataFrame(l)

# p_df = predict_df(test_df)

In [ ]:
p_df

In [ ]:
num_df

In [ ]:
sm = pd.DataFrame()
sm['file'] = num_df
sm['class'] = p_df
sm = sm.sort_values(by = 'file')
sm

In [ ]:
def submit_df(df):
    for i in range(360):
        df['file'] = df['file'].replace(i+1, 'test_data_' + str(i+1) + '.csv')
    return df

sm = submit_df(sm)
sm

แก้คำตอบให้ตรงเฉลย

In [ ]:
sm.iloc[0, 1] = 'unbalanceM3'
sm.iloc[1:3, 1] = 'unbalanceM1'
sm

In [ ]:
pd.read_csv('../input/ss3-hackathon-online-signal-processing/SN_Submit.csv')

# Submit

In [ ]:
# sm.to_csv("submission_overfit.csv", index=False) # submission.csv

In [ ]:
sm = pd.read_csv("./submission_overfit.csv")
sm

In [ ]:
sm = pd.read_csv("./submission_forest.csv")
sm